## Homework 04
  
The [Conversation AI](https://conversationai.github.io/) team, a research initiative founded by [Jigsaw](https://jigsaw.google.com/) and Google (both a part of Alphabet) are working on tools to help improve online conversation. One area of focus is the study of negative online behaviors, like toxic comments (i.e. comments that are rude, disrespectful or otherwise likely to make someone leave a discussion).   
  
In 2019, Kaggle hosted their [second competition](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge#description) on this research. The challenge was to create a model that is capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. The competitions used a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

We shall be using this dataset to benchmark a number of ML models. While the focus of the current competition is to mitigate bias, we will not be using the metric used in the competition. Instead we will be focusing on a simpler metric [Area under the Curve (or AUC)](https://www.kaggle.com/learn-forum/53782) which is suitable to unbalanced binary datasets. Also, we shall not consider different levels of Toxicity; we shall purely take anything marked over the 0.5 level in the measured toxicity range as toxic, and anything underneath as non-toxic. 

We have created a jupyter notbook with some of the tools to model this problem in Deep Learning, using Logistic regression and MLP. Your challenge will be to fill in the models and benchmark the AUC you achieve on these models.

We shall be using the keras deep learning package. As you may know, this is an API into DL frameworks, but is most commonly backed by Tensorflow. [keras.io](keras.io) is a great source for documentation and examples on layers available andn functionality. 

**Have fun!!**


*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

### Set up packages

In [ ]:
# Unfortunately the latest Keras version has a bug with Sparse matrices and we need to downgrade. 
!pip install tensorflow==1.14
!pip install keras==2.2.5
import os
import tensorflow as tf
import keras
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

     |████████████████████████████████| 109.3MB 79kB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.2MB 48.0MB/s 
     |████████████████████████████████| 491kB 38.9MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 337kB 5.4MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

### Load data
Let's load our data and take a peak.   
The competition metrics and fields have too much detail too cover here.   
We will just focus on the comment of the users and whether it was deemed toxic (target>0.5) or not.    

In [ ]:
# File with the training samples
!wget -O train.csv.zip "https://www.dropbox.com/s/xei6z41mfrcnxcd/train.csv.zip?dl=1"

--2021-03-15 17:10:36--  https://www.dropbox.com/s/xei6z41mfrcnxcd/train.csv.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/xei6z41mfrcnxcd/train.csv.zip [following]
--2021-03-15 17:10:36--  https://www.dropbox.com/s/dl/xei6z41mfrcnxcd/train.csv.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8ab640142c3805e7324bd23d18.dl.dropboxusercontent.com/cd/0/get/BKuXgcpFecorBqgu9ZflO-CZ7jXhoZ4MibqCXHf0-E6Domr2oBkvzHlg1E5hON9GvPmZ6KR_gMtGnanbKeRyP8aVNOlAGZa4gSihm7Dcea5AB6fjL1Obj-B20tXRjmM6qCzxaNT9_R7G6TPKC4vLJ36i/file?dl=1# [following]
--2021-03-15 17:10:37--  https://uc8ab640142c3805e7324bd23d18.dl.dropboxusercontent.com/cd/0/get/BKuXgcpFecorBqgu9ZflO-CZ7jXhoZ4MibqCXHf0-E6Domr2oBkvzHlg1E5hON9GvPmZ6KR_gMtGnanbKe

In [ ]:
# Lets look at what files we have available. 
os.listdir()

['.config', 'train.csv.zip', 'sample_data']

In [ ]:
train_df = pd.read_csv('train.csv.zip', 
                       dtype={'comment_text':str},
                       usecols=['comment_text', 'target'],
                       compression = 'zip')
train_df['target'] = (train_df['target']>0.5).astype(int)
print("Dataframe Number of (rows, columns) : {}".format(train_df.shape))

Dataframe Number of (rows, columns) : (1804874, 2)


In [ ]:
pd.options.display.max_colwidth = 200
train_df.head(10)

,target,comment_text
0,0,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!"
1,0,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!"
2,0,This is such an urgent design problem; kudos to you for taking it on. Very impressive!
3,0,Is this something I'll be able to install on my site? When will you be releasing it?
4,1,haha you guys are a bunch of losers.
5,1,ur a sh*tty comment.
6,0,hahahahahahahahhha suck it.
7,0,FFFFUUUUUUUUUUUUUUU
8,0,The ranchers seem motivated by mostly by greed; no one should have the right to allow their animals destroy public land.
9,0,It was a great show. Not a combo I'd of expected to be good together but it was.


### Create validation data set
Lets randomly 66/33 split the data into a training and validation set.   
**No change needed here - note, please do not change the KFold split parameters, keeping it consistent will help us debug.**

In [ ]:
cv = KFold(n_splits=3, shuffle=True, random_state=42)
trn_ids, val_ids = next(cv.split(train_df))
x_train, x_valid = train_df['comment_text'][trn_ids], train_df['comment_text'][val_ids]
y_train, y_valid = train_df['target'].values[trn_ids], train_df['target'].values[val_ids]

### Vectorize Count of tokens

![Count Vectorizer](https://cdn-images-1.medium.com/max/1600/1*LD5XaHzOnoniU4p137FL5g.jpeg)  
We shall start off performing some CPU based Deep Learning operations. Sparse matrices are better run on CPU.    
Do not underestimate CPU based Deep Learning such as MLP; these models can be very powerful and outperform complex much more complex DL models.   
Here we create a sparse matrix from the text with 200K of the most common unigram and bigrams.  
**Your task here is to convert the collection of text documents (found in the `comment_text` field) to a matrix of token counts.  
This can be done using the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) in scikit_learn.  
After creating the vectorizer, fit it based on the train matrix `x_train` and use this vectorizer to transform both the `x_train` and `x_valid` sets.   
Create sparse matrices called `X_trn_mat` and `X_val_mat`, and please call your vectorizer: `vectorizer`.  
Use the parameters max features = 200000 and the token pattern `\w+`. This token pattern matches one or more word characters (same as `[a-zA-Z0-9_]`) only. All other characters are stripped.
Also, we would like to count both unigrams and bigrams (pairs of words), so set the ngram range to `(1,2)`**

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Create a CountVectorizer, called `vectorizer`
# And create sparse matrices X_trn_mat & X_val_mat
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

vectorizer = CountVectorizer(analyzer='word', ngram_range = (1, 2), max_features = 200000, token_pattern = '\w+')

vectorizer = vectorizer.fit(x_train)
X_trn_mat = vectorizer.transform(x_train)
X_val_mat = vectorizer.transform(x_valid)

In [ ]:
print(vectorizer.get_feature_names()[:10])
print(vectorizer.get_feature_names()[100000:100000+10])

['0', '0 0', '0 00', '0 01', '0 05', '0 1', '0 2', '0 25', '0 3', '0 4']
['make more', 'make most', 'make much', 'make my', 'make new', 'make no', 'make noise', 'make obama', 'make of', 'make on']


In [ ]:
X_trn_mat

<1203249x200000 sparse matrix of type '<class 'numpy.int64'>'
	with 89432534 stored elements in Compressed Sparse Row format>

### Logistic Regression

![Logistic Regression](https://upload.wikimedia.org/wikipedia/commons/6/6d/Exam_pass_logistic_curve.jpeg)
  
Lets start off with a simple Logistic Regression, which is the very basic [sigmoid activation function](https://en.wikipedia.org/wiki/Sigmoid_function) used in DL.  
Notice we have no hidden layers, we take as input the whole sparse matrix, and as output the binary classifier prediction (`0<=output<=1`).  
The model has 200001 parameters. One coefficient per column in the sparse matrx, plus one bias variable - each of which is learned using gradient descent. 


In [ ]:
model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
out = keras.layers.Dense(1, activation='sigmoid')(model_in)
model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-2))
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 200001    
Total params: 200,001
Trainable params: 200,001
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_trn_mat, y_train, epochs=2, batch_size=2**13, validation_data=(X_val_mat, y_valid))

Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [==============================] - 9s 7us/step - loss: 0.2419 - val_loss: 0.1777
Epoch 2/2
1203249/1203249 [==============================] - 8s 7us/step - loss: 0.1332 - val_loss: 0.1489


In [ ]:
preds_lr = model.predict(X_val_mat).flatten()
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_lr)))

AUC score : 0.88790


Look at the coefficients to see which words are driving toxic and non-toxic sentences. 

In [ ]:
feats = np.array(vectorizer.get_feature_names())
importance_index = model.get_weights()[0].flatten().argsort()
print('Top toxic tokens : \n{}'.format(feats[importance_index[-10:]].tolist()))
print('\nTop non-toxic tokens : \n{}'.format(feats[importance_index[:10]].tolist()))

Top toxic tokens : 
['morons', 'damn', 'pathetic', 'hypocrite', 'shit', 'stupid', 'idiot', 'stupidity', 'idiots', 'idiotic']

Top non-toxic tokens : 
['well said', 'amen', 'bingo', 'obstacle', 'great comment', '1932', 'break any', 'and status', 'perhaps in', 'just talk']


### MLP

![MLP](https://www.researchgate.net/profile/Mouhammd_Alkasassbeh/publication/309592737/figure/fig2/AS:423712664100865@1478032379613/MultiLayer-Perceptron-MLP-sturcture-334-MultiLayer-Perceptron-Classifier-MultiLayer.png)

Here we shall create a Multi-layer perceptron. Although relatively simple, these can be very powerful models and also suited to low compute power. 
**Please add three hidden layers to the network using a `relu` activation function.  
You can refer to this [script](https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s). This script contains an MLP which took first place in the *Mercari Price Suggestion Challenge*.   
Note, you can do this task by only adding four lines of code. You should see a large increase in AUC over the Logistic Regression.**  
Never underestimate the power of an MLP!!

In [ ]:
model_in = keras.Input(shape=(X_trn_mat.shape[1],), dtype='float32', sparse=True)
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Please fill in the next lines with the three hidden layers and the output layer. 
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

out = keras.layers.Dense(100, activation='relu')(model_in)
out = keras.layers.Dense(100, activation='relu')(out)
out = keras.layers.Dense(100, activation='relu')(out)
out = keras.layers.Dense(1, activation='sigmoid')(out)

model = keras.Model(inputs=model_in, outputs=out)
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=1e-3))
model.summary()



Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 200000)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               20000100  
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 20,020,401
Trainable params: 20,020,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_trn_mat, y_train, batch_size=2**13, epochs=2, verbose=1, validation_data=(X_val_mat, y_valid))
preds_mlp = model.predict(X_val_mat).flatten()

Train on 1203249 samples, validate on 601625 samples
Epoch 1/2
1203249/1203249 [==============================] - 102s 85us/step - loss: 0.1989 - val_loss: 0.1202
Epoch 2/2
1203249/1203249 [==============================] - 100s 83us/step - loss: 0.0797 - val_loss: 0.1271


In [ ]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_mlp)))

AUC score : 0.92938


### MLP with regularisation

Now lets try regularization.  
**Copy the above MLP model and create a new one adding regularization into the MLP hidden layers.  
    Add l2 regularisation to each of the dense hidden layers. Check on [keras.io](https://keras.io) to find details on how to add l2 regularization. Play are around with different level of regularization to see when you achieve optimal results.   
Generally it is good to choose parameters like regularization by moving up and down in factors of `10`.  
Can you improve on your previous AUC results by using reglarisation?**